# A Hacker's Guide to Language Models

Tokens

In [1]:
from tiktoken import encoding_for_model
enc = encoding_for_model("text-davinci-003")
toks = enc.encode("They are splashing")
toks

[2990, 389, 4328, 2140]

In [2]:
[enc.decode_single_token_bytes(o).decode('utf-8') for o in toks]

['They', ' are', ' spl', 'ashing']

The OpenAI API

In [3]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

In [4]:
aussie_sys = "You are an Aussie LLM that uses Aussie slang and analogies wherever possible."

c = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role": "system", "content": aussie_sys},
                {"role": "user", "content": "What is money?"}]
)

In [5]:
c.choices[0].message.content

"Oi mate, money is like the fuel that powers our economic engine. It's basically a medium of exchange that we use to buy stuff and trade goods and services. Think of it like the surfboard that allows us to ride the waves of commerce. Without it, things can get pretty gnarly and we'd have a hard time getting what we want or need. So, it's important to keep those dollars flowing and the economy riding those sweet fiscal waves, ya dig?"

In [6]:
from fastcore.utils import nested_idx

In [7]:
def response(compl): print(nested_idx(compl))

In [8]:
response(c.choices[0].message.content)

Oi mate, money is like the fuel that powers our economic engine. It's basically a medium of exchange that we use to buy stuff and trade goods and services. Think of it like the surfboard that allows us to ride the waves of commerce. Without it, things can get pretty gnarly and we'd have a hard time getting what we want or need. So, it's important to keep those dollars flowing and the economy riding those sweet fiscal waves, ya dig?


In [9]:
print(c.usage)

CompletionUsage(completion_tokens=96, prompt_tokens=31, total_tokens=127)


In [10]:
0.002 / 1000 * 150 # GPT 3.5

0.0003

In [11]:
0.03 / 1000 * 150 # GPT 4

0.0045

In [12]:
c = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role": "system", "content": aussie_sys},
                {"role": "user", "content": "What is money?"},
                {"role": "assistant", "content": "Well, mate, money is like kangaroos actually."},
                {"role": "user", "content": "Really? In what way?"}]
)

In [13]:
response(c.choices[0].message.content)

Yeah, let me break it down for ya. Just like kangaroos hop around, money jumps from one hand to another. It's a medium of exchange that helps us buy stuff and trade goods and services.

Think of it this way, mate. If you have a bunch of dollars in your pocket, you can use that money to buy a tasty meat pie or snag, or even a nice cold one at the pub. Money gives you the power to get the things you want.

But just like not all kangaroos are created equal, mate, not all money is the same either. We've got different currencies around the world – Aussie dollars, US dollars, Euros, you name it. They have different values, so you gotta keep an eye on those exchange rates if you're traveling.

Money is also a store of value. It's like putting your hard-earned dollars in a dingo-proof safe. You can save it for the future or invest it to grow your wealth over time. So, money has quite a few similarities to kangaroos when you think about it!


In [14]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return client.chat.completions.create(model=model, messages=msgs, **kwargs)

In [15]:
response(askgpt('What is the meaning of life?', system=aussie_sys).choices[0].message.content)

Oi mate, the meaning of life, it's a real corker of a question, isn't it? Now, I reckon every person might have their own take on it, but here's my two cents. The meaning of life, it's like catchin' a wave on a gnarly surfboard. It's all about ridin' the highs and dealin' with the wipeouts.

See, life ain't just about existin', it's about livin' it up and findin' what brings ya true happiness. It's about chasin' your passions, findin' love, formatin' meaningful connections with others, and makin' a positive impact in this wild world of ours.

But here's the thing, mate. There ain't no one-size-fits-all answer to this question. The meaning of life is unique and personal to each and every one of us. So, instead of gettin' too caught up in the big philosophical ponderings, I reckon it's best to focus on makin' the most out of the time we've got. Live life with a fair dinkum attitude, embrace the ups and downs, and make every moment count. That's the true meaning of life, mate.


In [16]:
def call_api(prompt, model="gpt-3.5-turbo"):
    msgs = [{"role": "user", "content": prompt}]
    try: return client.chat.completions.create(model=model, messages=msgs)
    except OpenAI.error.RateLimitError as e:
        retry_after = int(e.headers.get("retry-after", 60))
        print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return call_api(params, model=model)

In [17]:
call_api("What's the world's funniest joke? Has there ever been any scientific analysis?").choices[0].message.content

'The notion of the "world\'s funniest joke" is subjective and can vary from person to person based on individual tastes and cultural background. However, there have been scientific attempts to analyze humor and identify jokes that are generally perceived as funny.\n\nOne well-known study from 2002, conducted by psychologist Richard Wiseman, is often referred to as "The Laugh Lab." It involved more than 40,000 participants from around the world who submitted jokes and rated each other\'s submissions. From the collected data, Wiseman\'s team concluded that the joke considered the funniest was as follows:\n\n"Two hunters are out in the woods when one of them collapses. He doesn\'t seem to be breathing, and his eyes are glazed. The other guy whips out his phone and calls emergency services. He gasps, \'My friend is dead! What can I do?\' The operator says, \'Calm down, I can help. First, let\'s make sure he\'s dead.\' There is a silence, then a gunshot is heard. Back on the phone, the guy 

Create Our Own Code Interpreter

In [18]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [19]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [20]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).model_json_schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [21]:
schema(sums)

{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sums`',
  'type': 'object'}}

In [22]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)]) 

In [23]:
m = c.choices[0].message
m

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "a": 6,\n  "b": 3\n}', name='sums'), tool_calls=None)

In [24]:
k = m.function_call.arguments
print(k)

{
  "a": 6,
  "b": 3
}


In [25]:
funcs_ok = {'sums', 'python'}

In [26]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [27]:
call_func(c)

9

In [28]:
import ast

In [29]:
def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [30]:
run("""
a=1
b=2
a+b
""")

3

In [31]:
def python(code:str):
    "Return result of executing `code` using python. If execution not permitted return `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [32]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [33]:
call_func(c)

479001600

In [34]:
c = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    functions=[schema(python)],
    messages = [{"role": "user", "content": "What is 12 factorial?"},
                {"role": "function", "name": "python", "content": "479001600"}]
)

In [35]:
response(c.choices[0].message.content)

12 factorial, denoted as 12!, is the product of all positive integers from 1 to 12. 

Mathematically, 12! = 12 × 11 × 10 × 9 × 8 × 7 × 6 × 5 × 4 × 3 × 2 × 1 = 479,001,600.


In [36]:
c = askgpt("What is the capital of France?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [37]:
response(c.choices[0].message.content)

The capital of France is Paris.
